<a href="https://colab.research.google.com/github/kHarshit/cuda-programming/blob/main/CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls /usr/local/

bin/    cuda@     cuda-12.2/  games/               include/  lib64/      man@   share/
colab/  cuda-12@  etc/        _gcs_config_ops.so/  lib/      licensing/  sbin/  src/


In [ ]:
%%bash
which nvcc
nvcc --version

/usr/local/cuda/bin/nvcc
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!nvidia-smi

Fri Jun  7 17:02:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp15169jho".


In [ ]:
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
}

Hello from block: 0, thread: 0
Hello from block: 0, thread: 1
Hello from block: 1, thread: 0
Hello from block: 1, thread: 1



## Vector addition

In [ ]:
%%cuda

#include <iostream>

/**
 * @brief Cuda kernel (c++ function) to add two vectors
 */
__global__ void vecAdd(float *A, float *B, float *C, int N)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < N)
        C[i] = A[i] + B[i];
}

// host code
int main()
{
    int N = 1024;
    size_t size = N * sizeof(float);

    // allocate input vectors in host memory
    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    // initialize h_A and h_B
    for (int i = 0; i < N; i++)
    {
        h_A[i] = i;
        h_B[i] = i + 1;
    }

    // allocate vectors in device memory
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    // copy vectors from host memory to device memory
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // kernel launch
    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;
    vecAdd<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // copy results from device memory to host memory
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // print results
    std::cout << "Result: First 5 values:" << std::endl;
    for (int i = 0; i < 5; i++)
        std::cout << h_A[i] << " + " << h_B[i] << " = " << h_C[i] << std::endl;

    // free host memory
    free(h_A);
    free(h_B);
    free(h_C);
}

Result: First 5 values:
0 + 1 = 1
1 + 2 = 3
2 + 3 = 5
3 + 4 = 7
4 + 5 = 9



## Matrix addition

In [ ]:
%%cuda
# include <cuda_runtime.h>
#include <iostream>

// CUDA kernel function
__global__ void MatAdd(float *A, float *B, float *C, int N)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
    int index = i * N + j;

    if (i < N && j < N)
        C[index] = A[index] + B[index];
}

int main()
{
    std::cout << "CUDA Matrix Addition" << std::endl;
    int N = 1024;
    std::cout << "Matrix size: " << N << std::endl;
    // float A[N][N], B[N][N], C[N][N];
    // Allocate memory for matrices on the host (heap allocation)
    float *A = new float[N * N];
    float *B = new float[N * N];
    float *C = new float[N * N];
    std::cout << "Initializing matrices..." << std::endl;
    // Initialize A and B with some values
    // Initialize A and B with some values
    for (int i = 0; i < N; ++i)
    {
        for (int j = 0; j < N; ++j)
        {
            A[i * N + j] = static_cast<float>(i * N + j);
            B[i * N + j] = static_cast<float>(j * N + i);
        }
    }

    float *d_A, *d_B, *d_C;
    std::cout << "Allocating memory on the GPU..." << std::endl;
    // Allocate memory on the GPU
    cudaMalloc((void**)&d_A, N * N * sizeof(float));
    cudaMalloc((void**)&d_B, N * N * sizeof(float));
    cudaMalloc((void**)&d_C, N * N * sizeof(float));
    std::cout << "Memory allocated on the GPU." << std::endl;
    // Flatten and copy data from host (CPU) to device (GPU)
    cudaMemcpy(d_A, A, N * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, N * N * sizeof(float), cudaMemcpyHostToDevice);
    std::cout << "Data copied to the GPU." << std::endl;
    // Define the number of threads per block and number of blocks
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (N + threadsPerBlock.y - 1) / threadsPerBlock.y);
    std::cout << "Launching the kernel..." << std::endl;
    // Launch the kernel
    MatAdd<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();
    std::cout << "Kernel launched." << std::endl;
    // Copy the result from device (GPU) to host (CPU)
    cudaMemcpy(C, d_C, N * N * sizeof(float), cudaMemcpyDeviceToHost);
    std::cout << "Result copied to the host." << std::endl;
    // Free memory on the GPU
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Print some of the results
    std::cout << "Result matrix C:" << std::endl;
    for (int i = 0; i < 5; ++i)
    {
        for (int j = 0; j < 5; ++j)
        {
            std::cout << C[i * N + j] << " ";
        }
        std::cout << std::endl;
    }

    return 0;
}


CUDA Matrix Addition
Matrix size: 1024
Initializing matrices...
Allocating memory on the GPU...
Memory allocated on the GPU.
Data copied to the GPU.
Launching the kernel...
Kernel launched.
Result copied to the host.
Result matrix C:
0 1025 2050 3075 4100 
1025 2050 3075 4100 5125 
2050 3075 4100 5125 6150 
3075 4100 5125 6150 7175 
4100 5125 6150 7175 8200 



## Matrix multiplication

Matrices are stored in row-major order:
```  
M(row, col) = *(M.elements + row * M.width + col)
```

## Naive Kernel

In [ ]:
%%cuda

#include <cuda_runtime.h>
#include <iostream>
#include <cstdio>

typedef struct {
    int width;
    int height;
    float *elements;
} Matrix;

// Thread block size
#define BLOCK_SIZE 16

// Forward declaration of matrix multiplication kernel
__global__ void matMulKernel(Matrix A, Matrix B, Matrix C) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Each thread accumulates one element of C by accumulating results into cValue
    float cValue = 0;

    // C[i][j] = sum_k A[i][k] * B[k][j]
    // Iterates over common dimensions of A and B (k = A.width = B.height)
    if (row < A.height && col < B.width) {
        for (int k = 0; k < A.width; ++k)
            cValue += A.elements[row * A.width + k] * B.elements[k * B.width + col];
        C.elements[row * C.width + col] = cValue;
    }
}

// Error checking macro
#define CUDA_CHECK_ERROR(call) \
do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        std::cerr << "CUDA error in " << __FILE__ << " at line " << __LINE__ << ": " \
                  << cudaGetErrorString(err) << std::endl; \
        std::exit(EXIT_FAILURE); \
    } \
} while (0)

/**
 * @brief Matrix multiplication (host code)
 */
void matMul(const Matrix A, const Matrix B, Matrix C) {
    // Load A, B to device memory
    Matrix d_A, d_B, d_C;

    size_t sizeA = A.width * A.height * sizeof(float);
    size_t sizeB = B.width * B.height * sizeof(float);
    size_t sizeC = C.width * C.height * sizeof(float);

    d_A.width = A.width; d_A.height = A.height;
    d_B.width = B.width; d_B.height = B.height;
    d_C.width = C.width; d_C.height = C.height;

    // Allocate device memory
    CUDA_CHECK_ERROR(cudaMalloc(&d_A.elements, sizeA));
    CUDA_CHECK_ERROR(cudaMalloc(&d_B.elements, sizeB));
    CUDA_CHECK_ERROR(cudaMalloc(&d_C.elements, sizeC));

    // Copy A, B to device memory
    CUDA_CHECK_ERROR(cudaMemcpy(d_A.elements, A.elements, sizeA, cudaMemcpyHostToDevice));
    CUDA_CHECK_ERROR(cudaMemcpy(d_B.elements, B.elements, sizeB, cudaMemcpyHostToDevice));

    // Launch kernel
    dim3 threadsPerBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 blocksPerGrid((B.width + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (A.height + threadsPerBlock.y - 1) / threadsPerBlock.y);
    matMulKernel<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C);

    // Synchronize device memory
    CUDA_CHECK_ERROR(cudaDeviceSynchronize());

    // Copy C from device memory to host memory
    CUDA_CHECK_ERROR(cudaMemcpy(C.elements, d_C.elements, sizeC, cudaMemcpyDeviceToHost));

    // Free device memory
    CUDA_CHECK_ERROR(cudaFree(d_A.elements));
    CUDA_CHECK_ERROR(cudaFree(d_B.elements));
    CUDA_CHECK_ERROR(cudaFree(d_C.elements));
}

int main()
{
    std::cout << "CUDA Matrix Multiplication" << std::endl;

    // A(A.height x A.width), B(B.height x B.width), C(C.height x C.width)
    Matrix A, B, C;
    A.width = B.height = 1024;
    A.height = 1024;
    B.width = 768;
    C.height = A.height;
    C.width = B.width;

    // Allocate memory for matrices on the host
    A.elements = new float[A.height * A.width];
    B.elements = new float[B.height * B.width];
    C.elements = new float[C.height * C.width];

    // Initialize matrices A and B with some values
    std::cout << "Initializing matrices..." << std::endl;
    for (int i = 0; i < A.height; ++i)
    {
        for (int j = 0; j < A.width; ++j)
        {
            A.elements[i * A.width + j] = static_cast<float>(i * A.width + j);
        }
    }

    for (int i = 0; i < B.height; ++i)
    {
        for (int j = 0; j < B.width; ++j)
        {
            B.elements[i * B.width + j] = static_cast<float>(j * B.width + i);
        }
    }

    // Call the matrix multiplication function
    std::cout << "Matrix multiplication..." << std::endl;
    matMul(A, B, C);

    // Print some of the results
    std::cout << "Result matrix C (first 5x5 elements):" << std::endl;
    for (int i = 0; i < 5; ++i)
    {
        for (int j = 0; j < 5; ++j)
        {
            std::cout << C.elements[i * C.width + j] << " ";
        }
        std::cout << std::endl;
    }

    // Free memory on the host
    delete[] A.elements;
    delete[] B.elements;
    delete[] C.elements;

    return 0;
}


CUDA Matrix Multiplication
Initializing matrices...
Matrix multiplication...
Result matrix C (first 5x5 elements):
3.57389e+08 7.5965e+08 1.16191e+09 1.56417e+09 1.96643e+09 
8.93736e+08 2.1013e+09 3.30887e+09 4.51644e+09 5.72401e+09 
1.43008e+09 3.44296e+09 5.45583e+09 7.46871e+09 9.48158e+09 
1.96643e+09 4.78461e+09 7.60279e+09 1.0421e+10 1.32392e+10 
2.50278e+09 6.12626e+09 9.74976e+09 1.33732e+10 1.69967e+10 



## Shared Memory

In [ ]:
%%cuda

#include <cuda_runtime.h>
#include <iostream>
#include <cstdio>

#define TILE_SIZE 16

typedef struct
{
    int width;
    int height;
    float *elements;
} Matrix;

// Kernel for matrix multiplication using tiling and shared memory
__global__ void matMulKernel(Matrix A, Matrix B, Matrix C)
{
    // Shared memory for tiles of A and B
    __shared__ float shared_A[TILE_SIZE][TILE_SIZE];
    __shared__ float shared_B[TILE_SIZE][TILE_SIZE];

    // Calculate the global row and column index of the element
    int globalRow = blockIdx.y * blockDim.y + threadIdx.y;
    int globalCol = blockIdx.x * blockDim.x + threadIdx.x;

    float Cvalue = 0.0f;

    // Thread row and column within Csub
    int row = threadIdx.y;
    int col = threadIdx.x;

    // Loop over the tiles of the input matrices
    // A.width/TILE_SIZE and B.height/TILE_SIZE; take care of the last tile
    for (int m = 0; m < (A.width + TILE_SIZE - 1) / TILE_SIZE; ++m)
    {
        // for dynamic array: shared_A[row * TILE_SIZE + col]
        if (row < A.height && (m * TILE_SIZE + col) < A.width) {
            shared_A[row][col] = A.elements[globalRow * A.width + m * TILE_SIZE + col];
        } else {
            // When matrix dimensions are not exact multiples of the tile size,
            // some threads in the last blocks might access elements outside
            // the matrix boundaries. By setting out-of-bounds elements to zero,
            // we ensure that these threads do not contribute invalid values to final result.
            // e.g. Matrix A = [100x100] and TILE_SIZE = 16
            shared_A[row][col] = 0.0f;
        }
        // Load elements of B into shared memory
        if (col < B.width && (m * TILE_SIZE + row) < B.height) {
            shared_B[row][col] = B.elements[(m * TILE_SIZE + row) * B.width + globalCol];
        } else {
            shared_B[row][col] = 0.0f;
        }
        // Synchronize to ensure all threads have loaded their elements
        __syncthreads();

        // Compute the partial result
        for (int k = 0; k < TILE_SIZE; ++k)
            Cvalue += shared_A[row][k] * shared_B[k][col];

        // Synchronize to ensure all threads have completed the computation
        __syncthreads();
    }

    // Write the result to global memory
    if (globalRow < C.height && globalCol < C.width)
        C.elements[globalRow * C.width + globalCol] = Cvalue;
}

// Error checking macro
#define CUDA_CHECK_ERROR(call) \
do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        std::cerr << "CUDA error in " << __FILE__ << " at line " << __LINE__ << ": " \
                  << cudaGetErrorString(err) << std::endl; \
        std::exit(EXIT_FAILURE); \
    } \
} while (0)

// Host function for matrix multiplication
void matMul(const Matrix A, const Matrix B, Matrix C)
{
    Matrix d_A, d_B, d_C;

    size_t size_A = A.width * A.height * sizeof(float);
    size_t size_B = B.width * B.height * sizeof(float);
    size_t size_C = C.width * C.height * sizeof(float);

    d_A.width = A.width; d_A.height = A.height;
    d_B.width = B.width; d_B.height = B.height;
    d_C.width = C.width; d_C.height = C.height;

    CUDA_CHECK_ERROR(cudaMalloc(&d_A.elements, size_A));
    CUDA_CHECK_ERROR(cudaMalloc(&d_B.elements, size_B));
    CUDA_CHECK_ERROR(cudaMalloc(&d_C.elements, size_C));

    CUDA_CHECK_ERROR(cudaMemcpy(d_A.elements, A.elements, size_A, cudaMemcpyHostToDevice));
    CUDA_CHECK_ERROR(cudaMemcpy(d_B.elements, B.elements, size_B, cudaMemcpyHostToDevice));

    dim3 threadsPerBlock(TILE_SIZE, TILE_SIZE);
    // B.width/TILE_SIZE and A.height/TILE_SIZE; take care of the last tile
    dim3 blocksPerGrid((C.width + TILE_SIZE - 1) / TILE_SIZE, (C.height + TILE_SIZE - 1) / TILE_SIZE);

    matMulKernel<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C);
    CUDA_CHECK_ERROR(cudaDeviceSynchronize());

    CUDA_CHECK_ERROR(cudaMemcpy(C.elements, d_C.elements, size_C, cudaMemcpyDeviceToHost));

    CUDA_CHECK_ERROR(cudaFree(d_A.elements));
    CUDA_CHECK_ERROR(cudaFree(d_B.elements));
    CUDA_CHECK_ERROR(cudaFree(d_C.elements));
}

int main()
{
    std::cout << "CUDA Matrix Multiplication with Coalescing" << std::endl;

    Matrix A, B, C;
    A.width = B.height = 1024;
    A.height = 1024;
    B.width = 768;
    C.height = A.height;
    C.width = B.width;

    A.elements = new float[A.height * A.width];
    B.elements = new float[B.height * B.width];
    C.elements = new float[C.height * C.width];

    std::cout << "Initializing matrices..." << std::endl;
    for (int i = 0; i < A.height; ++i)
        for (int j = 0; j < A.width; ++j)
            A.elements[i * A.width + j] = static_cast<float>(i * A.width + j);

    for (int i = 0; i < B.height; ++i)
        for (int j = 0; j < B.width; ++j)
            B.elements[i * B.width + j] = static_cast<float>(j * B.width + i);

    std::cout << "Performing matrix multiplication..." << std::endl;
    matMul(A, B, C);

    std::cout << "Result matrix C (first 5x5 elements):" << std::endl;
    for (int i = 0; i < 5; ++i)
    {
        for (int j = 0; j < 5; ++j)
        {
            std::cout << C.elements[i * C.width + j] << " ";
        }
        std::cout << std::endl;
    }

    delete[] A.elements;
    delete[] B.elements;
    delete[] C.elements;

    return 0;
}

CUDA Matrix Multiplication with Coalescing
Initializing matrices...
Performing matrix multiplication...
Result matrix C (first 5x5 elements):
3.57389e+08 7.5965e+08 1.16191e+09 1.56417e+09 1.96643e+09 
8.93736e+08 2.1013e+09 3.30887e+09 4.51644e+09 5.72401e+09 
1.43008e+09 3.44296e+09 5.45583e+09 7.46871e+09 9.48158e+09 
1.96643e+09 4.78461e+09 7.60279e+09 1.0421e+10 1.32392e+10 
2.50278e+09 6.12626e+09 9.74976e+09 1.33732e+10 1.69967e+10 



## Memory coalescing

In [ ]:
%%cuda

#include <cuda_runtime.h>
#include <iostream>
#include <cstdio>
#include <chrono>

#define TILE_SIZE 16

typedef struct {
    int width;
    int height;
    float *elements;
} Matrix;

// Function to initialize matrix elements
void initializeMatrix(Matrix &mat) {
    for (int i = 0; i < mat.height; ++i) {
        for (int j = 0; j < mat.width; ++j) {
            mat.elements[i * mat.width + j] = static_cast<float>(rand() % 100);
        }
    }
}

// Error checking macro
#define CUDA_CHECK_ERROR(call) \
do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        std::cerr << "CUDA error in " << __FILE__ << " at line " << __LINE__ << ": " \
                  << cudaGetErrorString(err) << std::endl; \
        std::exit(EXIT_FAILURE); \
    } \
} while (0)

__global__ void matMulNaiveKernel(Matrix A, Matrix B, Matrix C) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < A.height && col < B.width) {
        float cValue = 0.0f;
        for (int k = 0; k < A.width; ++k) {
            cValue += A.elements[row * A.width + k] * B.elements[k * B.width + col];
        }
        C.elements[row * C.width + col] = cValue;
    }
}

__global__ void transposeKernel(float* A, float* A_T, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        A_T[x * height + y] = A[y * width + x];
    }
}

__global__ void matMulTransposedAKernel(Matrix A_T, Matrix B, Matrix C) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < C.height && col < C.width) {
        float cValue = 0.0f;
        for (int k = 0; k < A_T.width; ++k) {
            cValue += A_T.elements[row * A_T.width + k] * B.elements[k * B.width + col];
        }
        C.elements[row * C.width + col] = cValue;
    }
}

void runTransposeKernel(float* A, float* A_T, int width, int height, dim3 gridDim, dim3 blockDim) {
    transposeKernel<<<gridDim, blockDim>>>(A, A_T, width, height);
    CUDA_CHECK_ERROR(cudaDeviceSynchronize());
}

__global__ void matMulCoalescedKernel(Matrix A, Matrix B, Matrix C) {
    const int x = blockIdx.x * TILE_SIZE + (threadIdx.x / TILE_SIZE);
    const int y = blockIdx.y * TILE_SIZE + (threadIdx.x % TILE_SIZE);

    if (x < A.height && y < B.width) {
        float cValue = 0.0f;
        for (int k = 0; k < A.width; ++k) {
            cValue += A.elements[x * A.width + k] * B.elements[k * B.width + y];
        }
        C.elements[x * C.width + y] = cValue;
    }
}

__global__ void matMulSharedMemoryKernel(Matrix A, Matrix B, Matrix C)
{
    // Shared memory for tiles of A and B
    __shared__ float shared_A[TILE_SIZE][TILE_SIZE];
    __shared__ float shared_B[TILE_SIZE][TILE_SIZE];

    // Calculate the global row and column index of the element
    int globalRow = blockIdx.y * blockDim.y + threadIdx.y;
    int globalCol = blockIdx.x * blockDim.x + threadIdx.x;

    float Cvalue = 0.0f;

    // Thread row and column within Csub
    int row = threadIdx.y;
    int col = threadIdx.x;

    // Loop over the tiles of the input matrices
    // A.width/TILE_SIZE and B.height/TILE_SIZE; take care of the last tile
    for (int m = 0; m < (A.width + TILE_SIZE - 1) / TILE_SIZE; ++m)
    {
        // for dynamic array: shared_A[row * TILE_SIZE + col]
        if (row < A.height && (m * TILE_SIZE + col) < A.width) {
            shared_A[row][col] = A.elements[globalRow * A.width + m * TILE_SIZE + col];
        } else {
            // When matrix dimensions are not exact multiples of the tile size,
            // some threads in the last blocks might access elements outside
            // the matrix boundaries. By setting out-of-bounds elements to zero,
            // we ensure that these threads do not contribute invalid values to final result.
            // e.g. Matrix A = [100x100] and TILE_SIZE = 16
            shared_A[row][col] = 0.0f;
        }
        // Load elements of B into shared memory
        if (col < B.width && (m * TILE_SIZE + row) < B.height) {
            shared_B[row][col] = B.elements[(m * TILE_SIZE + row) * B.width + globalCol];
        } else {
            shared_B[row][col] = 0.0f;
        }
        // Synchronize to ensure all threads have loaded their elements
        __syncthreads();

        // Compute the partial result
        for (int k = 0; k < TILE_SIZE; ++k)
            Cvalue += shared_A[row][k] * shared_B[k][col];

        // Synchronize to ensure all threads have completed the computation
        __syncthreads();
    }

    // Write the result to global memory
    if (globalRow < C.height && globalCol < C.width)
        C.elements[globalRow * C.width + globalCol] = Cvalue;
}

void runKernel(void(*kernel)(Matrix, Matrix, Matrix), const Matrix &A, const Matrix &B, Matrix &C, dim3 gridDim, dim3 blockDim) {
    Matrix d_A, d_B, d_C;

    size_t size_A = A.width * A.height * sizeof(float);
    size_t size_B = B.width * B.height * sizeof(float);
    size_t size_C = C.width * C.height * sizeof(float);

    d_A.width = A.width; d_A.height = A.height;
    d_B.width = B.width; d_B.height = B.height;
    d_C.width = C.width; d_C.height = C.height;

    CUDA_CHECK_ERROR(cudaMalloc(&d_A.elements, size_A));
    CUDA_CHECK_ERROR(cudaMalloc(&d_B.elements, size_B));
    CUDA_CHECK_ERROR(cudaMalloc(&d_C.elements, size_C));

    CUDA_CHECK_ERROR(cudaMemcpy(d_A.elements, A.elements, size_A, cudaMemcpyHostToDevice));
    CUDA_CHECK_ERROR(cudaMemcpy(d_B.elements, B.elements, size_B, cudaMemcpyHostToDevice));

    auto start = std::chrono::high_resolution_clock::now();

    kernel<<<gridDim, blockDim>>>(d_A, d_B, d_C);

    CUDA_CHECK_ERROR(cudaDeviceSynchronize());

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<float> duration = end - start;
    std::cout << "Kernel execution time: " << duration.count() * 1000.0f << " ms" << std::endl;

    CUDA_CHECK_ERROR(cudaMemcpy(C.elements, d_C.elements, size_C, cudaMemcpyDeviceToHost));

    CUDA_CHECK_ERROR(cudaFree(d_A.elements));
    CUDA_CHECK_ERROR(cudaFree(d_B.elements));
    CUDA_CHECK_ERROR(cudaFree(d_C.elements));
}

int main() {
    std::cout << "CUDA Matrix Multiplication Comparison" << std::endl;

    Matrix A, B, C;
    A.width = B.height = 1024;
    A.height = 1024;
    B.width = 768;
    C.height = A.height;
    C.width = B.width;

    A.elements = new float[A.height * A.width];
    B.elements = new float[B.height * B.width];
    C.elements = new float[C.height * C.width];

    initializeMatrix(A);
    initializeMatrix(B);

    dim3 gridDim((C.width + TILE_SIZE - 1) / TILE_SIZE, (C.height + TILE_SIZE - 1) / TILE_SIZE);
    dim3 blockDim(TILE_SIZE, TILE_SIZE);

    std::cout << "Running naive kernel..." << std::endl;
    runKernel(matMulNaiveKernel, A, B, C, gridDim, blockDim);

    std::cout << "Running coalesced kernel..." << std::endl;
    dim3 coalescedBlockDim(TILE_SIZE * TILE_SIZE);
    runKernel(matMulCoalescedKernel, A, B, C, gridDim, coalescedBlockDim);

    std::cout << "Running shared memory kernel..." << std::endl;
    runKernel(matMulSharedMemoryKernel, A, B, C, gridDim, blockDim);


    delete[] A.elements;
    delete[] B.elements;
    delete[] C.elements;

    return 0;
}


CUDA Matrix Multiplication Comparison
Running naive kernel...
Kernel execution time: 75.8888 ms
Running coalesced kernel...
Kernel execution time: 5.15341 ms
Running shared memory kernel...
Kernel execution time: 4.43372 ms



### Transpose vs 2d grid

You can also do memory coalescing of A by taking its transpose compared to above creation of 2d grids of threads.

In [ ]:
%%cuda

#include <cuda_runtime.h>
#include <iostream>
#include <cstdio>
#include <chrono>

#define TILE_SIZE 16

typedef struct {
    int width;
    int height;
    float *elements;
} Matrix;

// Function to initialize matrix elements
void initializeMatrix(Matrix &mat) {
    for (int i = 0; i < mat.height; ++i) {
        for (int j = 0; j < mat.width; ++j) {
            mat.elements[i * mat.width + j] = static_cast<float>(rand() % 100);
        }
    }
}

// Naive matrix multiplication kernel
__global__ void matMulNaiveKernel(Matrix A, Matrix B, Matrix C) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < A.height && col < B.width) {
        float cValue = 0.0f;
        for (int k = 0; k < A.width; ++k) {
            cValue += A.elements[row * A.width + k] * B.elements[k * B.width + col];
        }
        C.elements[row * C.width + col] = cValue;
    }
}

// Matrix multiplication kernel with coalesced access using specific indexing
__global__ void matMulCoalescedKernel(Matrix A, Matrix B, Matrix C) {
    const int x = blockIdx.x * TILE_SIZE + (threadIdx.x / TILE_SIZE);
    const int y = blockIdx.y * TILE_SIZE + (threadIdx.x % TILE_SIZE);

    if (x < A.height && y < B.width) {
        float cValue = 0.0f;
        for (int k = 0; k < A.width; ++k) {
            cValue += A.elements[x * A.width + k] * B.elements[k * B.width + y];
        }
        C.elements[x * C.width + y] = cValue;
    }
}

// Transpose kernel for matrix A
__global__ void transposeKernel(float* A, float* A_T, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        A_T[x * height + y] = A[y * width + x];
    }
}

// Matrix multiplication kernel using transposed A
__global__ void matMulTransposedAKernel(Matrix A_T, Matrix B, Matrix C) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < C.height && col < C.width) {
        float cValue = 0.0f;
        for (int k = 0; k < A_T.height; ++k) {
            cValue += A_T.elements[col * A_T.width + k] * B.elements[k * B.width + row];
        }
        C.elements[row * C.width + col] = cValue;
    }
}

// Error checking macro
#define CUDA_CHECK_ERROR(call) \
do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        std::cerr << "CUDA error in " << __FILE__ << " at line " << __LINE__ << ": " \
                  << cudaGetErrorString(err) << std::endl; \
        std::exit(EXIT_FAILURE); \
    } \
} while (0)

void runTransposeKernel(float* A, float* A_T, int width, int height, dim3 gridDim, dim3 blockDim) {
    transposeKernel<<<gridDim, blockDim>>>(A, A_T, width, height);
    CUDA_CHECK_ERROR(cudaDeviceSynchronize());
}

void runMatMulKernel(void(*kernel)(Matrix, Matrix, Matrix), const Matrix &A, const Matrix &B, Matrix &C, dim3 gridDim, dim3 blockDim) {
    Matrix d_A, d_B, d_C;

    size_t size_A = A.width * A.height * sizeof(float);
    size_t size_B = B.width * B.height * sizeof(float);
    size_t size_C = C.width * C.height * sizeof(float);

    d_A.width = A.width; d_A.height = A.height;
    d_B.width = B.width; d_B.height = B.height;
    d_C.width = C.width; d_C.height = C.height;

    CUDA_CHECK_ERROR(cudaMalloc(&d_A.elements, size_A));
    CUDA_CHECK_ERROR(cudaMalloc(&d_B.elements, size_B));
    CUDA_CHECK_ERROR(cudaMalloc(&d_C.elements, size_C));

    CUDA_CHECK_ERROR(cudaMemcpy(d_A.elements, A.elements, size_A, cudaMemcpyHostToDevice));
    CUDA_CHECK_ERROR(cudaMemcpy(d_B.elements, B.elements, size_B, cudaMemcpyHostToDevice));

    auto start = std::chrono::high_resolution_clock::now();

    kernel<<<gridDim, blockDim>>>(d_A, d_B, d_C);

    CUDA_CHECK_ERROR(cudaDeviceSynchronize());

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<float> duration = end - start;
    std::cout << "Kernel execution time: " << duration.count() * 1000.0f << " ms" << std::endl;

    CUDA_CHECK_ERROR(cudaMemcpy(C.elements, d_C.elements, size_C, cudaMemcpyDeviceToHost));

    CUDA_CHECK_ERROR(cudaFree(d_A.elements));
    CUDA_CHECK_ERROR(cudaFree(d_B.elements));
    CUDA_CHECK_ERROR(cudaFree(d_C.elements));
}

int main() {
    std::cout << "CUDA Matrix Multiplication Comparison" << std::endl;

    Matrix A, B, C;
    A.width = B.height = 1024;
    A.height = 1024;
    B.width = 768;
    C.height = A.height;
    C.width = B.width;

    A.elements = new float[A.height * A.width];
    B.elements = new float[B.height * B.width];
    C.elements = new float[C.height * C.width];

    initializeMatrix(A);
    initializeMatrix(B);

    // Naive kernel
    dim3 gridDim((C.width + TILE_SIZE - 1) / TILE_SIZE, (C.height + TILE_SIZE - 1) / TILE_SIZE);
    dim3 blockDim(TILE_SIZE, TILE_SIZE);

    std::cout << "Running naive kernel..." << std::endl;
    runMatMulKernel(matMulNaiveKernel, A, B, C, gridDim, blockDim);

    // Transpose A
    Matrix A_T;
    A_T.width = A.height;
    A_T.height = A.width;
    A_T.elements = new float[A_T.width * A_T.height];

    float* d_A;
    float* d_A_T;
    CUDA_CHECK_ERROR(cudaMalloc(&d_A, A.width * A.height * sizeof(float)));
    CUDA_CHECK_ERROR(cudaMalloc(&d_A_T, A_T.width * A_T.height * sizeof(float)));
    CUDA_CHECK_ERROR(cudaMemcpy(d_A, A.elements, A.width * A.height * sizeof(float), cudaMemcpyHostToDevice));

    dim3 transposeGridDim((A.width + TILE_SIZE - 1) / TILE_SIZE, (A.height + TILE_SIZE - 1) / TILE_SIZE);
    dim3 transposeBlockDim(TILE_SIZE, TILE_SIZE);

    runTransposeKernel(d_A, d_A_T, A.width, A.height, transposeGridDim, transposeBlockDim);
    CUDA_CHECK_ERROR(cudaMemcpy(A_T.elements, d_A_T, A_T.width * A_T.height * sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "Running matrix multiplication with transposed A..." << std::endl;
    runMatMulKernel(matMulTransposedAKernel, A_T, B, C, gridDim, blockDim);

    // Coalesced kernel with specific indexing
    std::cout << "Running coalesced kernel with specific indexing..." << std::endl;
    dim3 coalescedBlockDim(TILE_SIZE * TILE_SIZE);
    runMatMulKernel(matMulCoalescedKernel, A, B, C, gridDim, coalescedBlockDim);

    CUDA_CHECK_ERROR(cudaFree(d_A));
    CUDA_CHECK_ERROR(cudaFree(d_A_T));

    delete[] A.elements;
    delete[] B.elements;
    delete[] A_T.elements;
    delete[] C.elements;

    return 0;
}


CUDA Matrix Multiplication Comparison
Running naive kernel...
Kernel execution time: 7.04247 ms
Running matrix multiplication with transposed A...
Kernel execution time: 36.7824 ms
Running coalesced kernel with specific indexing...
Kernel execution time: 5.15543 ms



# cuBLAS

In [ ]:
%%cuda

#include <cublas_v2.h>
#include <cuda_runtime.h>
#include <iostream>

void cuBLASMatrixMultiplication(const float *A, const float *B, float *C, int M, int N, int K) {
    cublasHandle_t handle;
    cublasCreate(&handle);

    const float alpha = 1.0f;
    const float beta = 0.0f;

    // Perform the matrix multiplication: C = alpha * A * B + beta * C
    cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, N, M, K, &alpha, B, N, A, K, &beta, C, N);

    cublasDestroy(handle);
}

int main() {
    int M = 1024, N = 768, K = 1024;
    size_t sizeA = M * K * sizeof(float);
    size_t sizeB = K * N * sizeof(float);
    size_t sizeC = M * N * sizeof(float);

    float *h_A = (float*)malloc(sizeA);
    float *h_B = (float*)malloc(sizeB);
    float *h_C = (float*)malloc(sizeC);

    // Initialize matrices h_A and h_B
    for (int i = 0; i < M * K; ++i) h_A[i] = static_cast<float>(rand() % 100);
    for (int i = 0; i < K * N; ++i) h_B[i] = static_cast<float>(rand() % 100);

    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, sizeA);
    cudaMalloc(&d_B, sizeB);
    cudaMalloc(&d_C, sizeC);

    cudaMemcpy(d_A, h_A, sizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, sizeB, cudaMemcpyHostToDevice);

    cuBLASMatrixMultiplication(d_A, d_B, d_C, M, N, K);

    cudaMemcpy(h_C, d_C, sizeC, cudaMemcpyDeviceToHost);

    // Print some elements of the result
    for (int i = 0; i < 10; ++i) {
        std::cout << h_C[i] << " ";
    }
    std::cout << std::endl;

    free(h_A);
    free(h_B);
    free(h_C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


/usr/bin/ld: /tmp/tmpxft_0000523c_00000000-11_single_file.o: in function `cuBLASMatrixMultiplication(float const*, float const*, float*, int, int, int)':
tmpxft_0000523c_00000000-6_single_file.cudafe1.cpp:(.text+0x54): undefined reference to `cublasCreate_v2'
/usr/bin/ld: tmpxft_0000523c_00000000-6_single_file.cudafe1.cpp:(.text+0xb0): undefined reference to `cublasSgemm_v2'
/usr/bin/ld: tmpxft_0000523c_00000000-6_single_file.cudafe1.cpp:(.text+0xc0): undefined reference to `cublasDestroy_v2'
collect2: error: ld returned 1 exit status

